In [143]:
import torch
from IPython import display
from d2l import torch as d2l

batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

## 初始化参数

In [144]:
num_inputs = 784  #28*28
num_output = 10
W = torch.normal(0,0.01,size=[num_inputs,num_output],requires_grad=True)
b = torch.zeros(num_output,requires_grad=True)

## 定义softmax函数

In [145]:
def softmax(X):
    X_exp = torch.exp(X)    #size = [x,y]
    partition = X_exp.sum(1, keepdim=True) #按列求和 #size = [1,y]
    return X_exp/partition #利用了广播机制  size = [x,y]

In [146]:
#测试softmax函数
X = torch.normal(0,1,(2,5))
X_prob = softmax(X)
print(X_prob, X_prob.sum(1))

tensor([[0.0356, 0.3755, 0.2099, 0.0644, 0.3146],
        [0.1370, 0.1795, 0.3829, 0.0313, 0.2694]]) tensor([1.0000, 1.0000])


# 定义模型线性回归
y = soft(WX+b)

In [147]:
def net(X):
    return softmax(torch.matmul(X.reshape((-1, W.shape[0])), W) + b)

In [148]:
# y = torch.tensor([0, 2])
# y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
# y_hat[[0, 1], y]  #返回[0,0] [1,2]两个位置的数据

In [149]:
#返回预测的真实标签类的交叉熵损失
def cross_entropy(y_hat, y):
    return - torch.log(y_hat[range(len(y_hat)), y])
# cross_entropy(y_hat, y)         #

## 分类的准确率计算

In [150]:
def accuracy(y_hat, y):  #@save
    """计算预测正确的数量"""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

In [151]:
# accuracy(y_hat, y) / len(y)

In [152]:
class Accumulator:  #@save
    """在n个变量上累加"""
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [153]:
def evaluate_accuracy(net, data_iter):  #@save
    """计算在指定数据集上模型的精度"""
    if isinstance(net, torch.nn.Module):
        net.eval()  # 将模型设置为评估模式
    metric = Accumulator(2)  # 正确预测数、预测总数
    with torch.no_grad():
        for X, y in data_iter:
            #accuracy(net(X), y)，每一批次正确的数量
            #每一批的总数
            metric.add(accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]

In [154]:
evaluate_accuracy(net, test_iter)

0.0507

## 训练

In [155]:
def train_epoch_ch3(net, train_iter, loss, updater):  #@save
    """训练模型一个迭代周期（定义见第3章）"""
    # 将模型设置为训练模式
    if isinstance(net, torch.nn.Module):
        net.train()
    # 训练损失总和、训练准确度总和、样本数
    metric = Accumulator(3)
    for X, y in train_iter:
        # 计算梯度并更新参数
        y_hat = net(X)
        l = loss(y_hat, y)
        if isinstance(updater, torch.optim.Optimizer):
            # 使用PyTorch内置的优化器和损失函数
            updater.zero_grad()
            l.mean().backward()
            updater.step()
        else:
            # 使用定制的优化器和损失函数
            l.sum().backward()
            updater(X.shape[0])
        metric.add(float(l.sum()), accuracy(y_hat, y), y.numel())
    # 返回训练损失和训练精度
    return metric[0] / metric[2], metric[1] / metric[2]


